In [1]:
import requests
import json
from datetime import datetime
import pandas as pd
import numpy as np

In [2]:
api_key = "2a12e95ec0d648259b2598674b29b743"
ip = 'https://newsapi.org/v2/'
types = ['top-headlines', 'everything']
sources = ['bloomberg', 'cnn', 'financial-times']

target_companies = set()
with open('company_names.txt') as f:
    for line in f:
        target_companies.update(c.strip() for c in line.split(', '))

In [3]:
from datetime import date, timedelta
date = date.today()
days = 30 # max 30 for free api

In [16]:
# document: https://newsapi.org/docs/endpoints/everything
page_size = 100
pages = 99

def fetch_data(date, days):
    news = []
    for i in range(days):
        date = date + timedelta(-1)
        date_string = date.strftime("%Y-%m-%d")
        sdate = date + timedelta(-1)
        sdate_string = sdate.strftime("%Y-%m-%d")
        print("date_string: ",date_string)
        print("sdate_string: ",sdate_string)
        news_of_date = []
        for source in sources:
            url = "{ip}{types}?sources={source}&to={edate}&pageSize={page_size}&page={pages}&apiKey={api_key}".format(
                ip=ip, types=types[1], #sdate=sdate_string, 
                edate=date_string, source=source,
                page_size=page_size, pages=pages,
                api_key=api_key)
            response = requests.get(url)
            try:
                news_of_date.extend(response.json()['articles'])
            except:
                print(response.json())
        print('number of news this day:{}\n'.format(len([t["description"] for t in news_of_date])))
        news.extend(news_of_date)
    return news

In [23]:
news = fetch_data(date.today(), days)

In [18]:
# number_news = page_size * pages
# news = []
# i = 0
# while i < number_news:
#     i += 200
#     fetch_data()

In [20]:
texts = [t["description"] for t in news]

In [21]:
len(texts)

800

In [22]:
len(set(texts))

785

In [16]:
def find_shared_word(str1, set1):
    wd_set1 = set(str1.lower().split())
    return wd_set1 & set1

In [17]:
def parse_time(raw_time):
    date_format = '%Y-%m-%dT%H:%M:%SZ'
    return datetime.strptime(raw_time, date_format)

In [18]:
def info_extractor():
    time, title, description, url = zip(*[(parse_time(elem['publishedAt']), elem['title'], elem['description'], elem['url']) 
                                          for elem in news if find_shared_word(elem['title'], target_companies)])
    df = pd.DataFrame(np.column_stack([time, title, description, url]), columns=['time', 'title', 'description', 'url'])
    return df

In [13]:
pd.set_option("max_colwidth", 300)

In [14]:
info_extractor()

,time,title,description,url
0,2018-05-02 23:25:49,"Amazon Launches Its Own Pet Product Brand, Wag; Starts With Food","Amazon.com Inc. has started its own brand of pet products, called Wag, expanding on existing private-label efforts that include batteries, baby products, clothing and household goods.",https://www.bloomberg.com/news/articles/2018-05-02/amazon-launches-its-own-pet-product-brand-wag-starts-with-food


In [ ]:
positive_v = ['is launching', 'reaches .. high', 'expansion', 'new', 'popular']
negative_v = ['lawsuit', 'claim agaist', 'scam', 'fraud', 'drop', 'problem']